In [ ]:
import pandas as pd
from scipy.stats import f_oneway

# Cargar datos
data = pd.read_csv("../files/input/experiment_data.csv")

# ANOVA
f_stat, p_value = f_oneway(data["Absorbance_Exp1"], data["Absorbance_Exp2"], data["Absorbance_Exp3"])

# Resultados
print(f"F-statistic: {f_stat}, p-value: {p_value}")
print("Conclusión:", "Diferencias significativas" if p_value < 0.05 else "No hay diferencias significativas")


In [ ]:
import numpy as np
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

# Reformatear los datos para suavizado
data_melted = data.melt(id_vars=["Time (min)"], var_name="Experiment", value_name="Absorbance")
data_melted["Experiment"] = data_melted["Experiment"].str.extract(r'(\d)').astype(int)

# Configuración para el suavizado
n_corridas = 3
n_puntos_por_corrida = len(data_melted) // n_corridas
window_length = 20  # Longitud de la ventana (debe ser impar)
polyorder = 2  # Orden del polinomio

# Reestructurar los datos en corridas
data_melted["Group"] = data_melted.groupby("Experiment").cumcount() // n_puntos_por_corrida

# Aplicar el filtro Savitzky-Golay para cada corrida
data_melted["Smoothed"] = data_melted.groupby(["Experiment", "Group"])["Absorbance"].transform(
    lambda x: savgol_filter(x, window_length, polyorder) if len(x) >= window_length else x
)

# Calcular el promedio suavizado
promedio_suavizado = data_melted.groupby("Time (min)")["Smoothed"].mean().reset_index()
promedio_suavizado.rename(columns={"Smoothed": "Average Smoothed"}, inplace=True)

# Graficar datos originales y suavizados
plt.figure(figsize=(12, 8))

# Graficar cada experimento original y suavizado
for exp in data_melted["Experiment"].unique():
    exp_data = data_melted[data_melted["Experiment"] == exp]
    plt.plot(exp_data["Time (min)"], exp_data["Absorbance"], 'o', alpha=0.5, label=f'Exp {exp} (Original)')
    plt.plot(exp_data["Time (min)"], exp_data["Smoothed"], '-', label=f'Exp {exp} (Suavizado)')

# Graficar promedio suavizado
plt.plot(promedio_suavizado["Time (min)"], promedio_suavizado["Average Smoothed"], 'k--', linewidth=2, label='Promedio Suavizado')

# Configuración de la gráfica
plt.title('Promedio Suavizado de Datos de Absorbancia con Savitzky-Golay (Pandas)')
plt.xlabel('Tiempo (min)')
plt.ylabel('Absorbancia (u.a.)')
plt.legend()
plt.grid(True)
plt.show()


In [29]:
import os

# Ruta del directorio de salida
output_dir = "../files/output"

# Verificar si el directorio de salida existe, si no, crearlo
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Guardar únicamente tiempo y promedio suavizado en el DataFrame final
promedio_suavizado.to_csv(os.path.join(output_dir, "data_clean.csv"), index=False)


In [ ]:
from scipy.optimize import curve_fit
import pickle
import os
import pandas as pd

# Crear el directorio de modelos si no existe
models_dir = "../files/models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    print(f"Directorio creado: {models_dir}")

# Cargar los datos experimentales
df = pd.read_csv("../files/output/data_clean.csv")
t = df["Time (min)"].values
A = df["Average Smoothed"].values

n_train = 13  # Número de puntos para el ajuste
el = 1416.536  # Absortividad molar por cm

In [ ]:
# Definir el modelo
def model_1(t, Ag, k, p_td):
    return el * (p_td * np.exp(-k * t) + Ag * (1 - np.exp(-k * t)))

# Ajustar el modelo
t_fit = t[:n_train]
A_fit = A[:n_train]

initial_guess = [0.0018, 1, 0.00018]
bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])

params, covariance = curve_fit(model_1, t_fit, A_fit, p0=initial_guess, bounds=bounds)

# Guardar los parámetros y el modelo en el Pickle
model_data = {
    "parameters": params,
    "covariance": covariance,
    "model": model_1  # Incluir la función del modelo
}

model_path = os.path.join(models_dir, "model_1.pkl")
with open(model_path, "wb") as file:
    pickle.dump(model_data, file)

print(f"Modelo guardado en: {model_path}")
print("Parámetros ajustados:", params)

In [ ]:
# Definir el modelo
def model_2(t, p_td, theta, k):
    return el * (p_td + theta * (1 - np.exp(-k * t)))

# Ajustar el modelo
t_fit = t[:n_train]
A_fit = A[:n_train]

initial_guess = [0.0018, 1, 0.00018]
bounds = ([0, 0, 0], [np.inf, np.inf, np.inf])

params, covariance = curve_fit(model_2, t_fit, A_fit, p0=initial_guess, bounds=bounds)

# Guardar los parámetros y el modelo en el Pickle
model_data = {
    "parameters": params,
    "covariance": covariance,
    "model": model_2  # Incluir la función del modelo
}

model_path = os.path.join(models_dir, "model_2.pkl")
with open(model_path, "wb") as file:
    pickle.dump(model_data, file)

print(f"Modelo guardado en: {model_path}")
print("Parámetros ajustados:", params)


In [ ]:
# Definir el modelo
def model_3(t, beta, k, x1, y1):
    return el * (x1 + y1 * np.log(np.abs(1 + beta) / (np.exp(-k * t) + beta)))

# Ajustar el modelo
t_fit = t[:n_train]
A_fit = A[:n_train]

initial_guess = [2.81547, 0.02725, 9.0226e-05, 0.00416]
bounds = ([2, 0.02, 0, 0], [3, 0.02725, 9.0226e-05, 0.00416])

params, covariance = curve_fit(model_3, t_fit, A_fit, p0=initial_guess, bounds=bounds)

# Guardar los parámetros y el modelo en el Pickle
model_data = {
    "parameters": params,
    "covariance": covariance,
    "model": model_3  # Incluir la función del modelo
}

model_path = os.path.join(models_dir, "model_3.pkl")
with open(model_path, "wb") as file:
    pickle.dump(model_data, file)

print(f"Modelo guardado en: {model_path}")
print("Parámetros ajustados:", params)


In [ ]:
from scipy.special import lambertw

# Definir el modelo 4
def model_4(t, c1, k, x1, y1):
    epsilon = 1416.536
    return epsilon * (x1 - y1 * np.real(lambertw(c1 * np.exp(-k * t))))

# Ajustar el modelo
t_fit = t[:n_train]
A_fit = A[:n_train]

initial_guess = [0.15, 0.027, 0.001356, 0.00839]
bounds = ([0, 0.026, 0.001355, 0.00838], [0.3, 0.028, 0.001357, 0.00840])  # Margen ajustado

params, covariance = curve_fit(model_4, t_fit, A_fit, p0=initial_guess, bounds=bounds)

# Guardar los parámetros y el modelo en el Pickle
model_data = {
    "parameters": params,
    "covariance": covariance,
    "model": model_4  # Incluir la función del modelo
}

model_path = os.path.join(models_dir, "model_4.pkl")
with open(model_path, "wb") as file:
    pickle.dump(model_data, file)

print(f"Modelo guardado en: {model_path}")
print("Parámetros ajustados:", params)



In [35]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
from sklearn.metrics import r2_score

def graf_and_save(model_pickle_path, data_path, output_dir, n_train):
    """
    Genera una gráfica personalizada entre datos experimentales, modelo ajustado y predicciones,
    y muestra el valor de R^2 directamente en la gráfica.
    
    Parameters:
        model_pickle_path (str): Ruta al archivo Pickle con el modelo ajustado.
        data_path (str): Ruta al archivo CSV con los datos experimentales.
        output_dir (str): Directorio donde se guardará la gráfica.
        n_train (int): Número de puntos utilizados para entrenar el modelo.
    """
    # Crear el directorio de salida si no existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Directorio creado: {output_dir}")

    # Identificar el nombre del modelo a partir del archivo Pickle
    model_name = os.path.splitext(os.path.basename(model_pickle_path))[0].replace("_", " ").capitalize()

    # Cargar el modelo desde el archivo Pickle
    with open(model_pickle_path, "rb") as file:
        model_data = pickle.load(file)
    params = model_data["parameters"]
    model = model_data["model"]

    # Cargar los datos experimentales
    df = pd.read_csv(data_path)
    t = df["Time (min)"].values
    A = df["Average Smoothed"].values

    # Separar datos de entrenamiento y predicción
    t_train = t[:n_train]
    A_train = A[:n_train]
    t_predict = t[n_train:]
    A_predict = A[n_train:]

    # Predicciones del modelo
    A_model = model(t, *params)

    # Calcular R^2
    r2 = r2_score(A, A_model)

    # Crear la gráfica
    plt.figure(figsize=(12, 8))

    # Colores personalizados
    colors = {
        "Modelo ajustado": "#0077b6",  # Azul oscuro
        "Datos entrenamiento": "#ff006e",  # Fucsia
        "Datos predicción": "#ffca3a"  # Amarillo pastel
    }

    # Graficar modelo ajustado en todo el rango
    plt.plot(
        t, A_model,
        color=colors["Modelo ajustado"],
        label="Modelo ajustado",
        linewidth=3,
        zorder=2,
    )

    # Graficar datos de entrenamiento
    plt.scatter(
        t_train, A_train,
        color=colors["Datos entrenamiento"],
        label="Datos de entrenamiento",
        s=80,
        alpha=0.9,
        zorder=3,
    )

    # Graficar datos de predicción
    plt.scatter(
        t_predict, A_predict,
        color=colors["Datos predicción"],
        label="Datos de predicción",
        s=80,
        alpha=0.9,
        zorder=3,
    )

    # Mostrar R^2 directamente en la gráfica
    plt.text(
        0.05, 0.95, f"$R^2 = {r2:.4f}$",
        fontsize=16,
        color="black",
        transform=plt.gca().transAxes,
        ha="left", va="top",
        bbox=dict(boxstyle="round,pad=0.3", edgecolor="gray", facecolor="white")
    )

    # Personalización de la gráfica
    plt.title(f"{model_name}", fontsize=20, fontweight="bold", color="#023047")
    plt.suptitle("Visualización de Entrenamiento y Predicción", fontsize=14, color="gray")
    plt.xlabel("Tiempo (min)", fontsize=14, color="#6c757d")
    plt.ylabel("Absorbancia (u.a.)", fontsize=14, color="#6c757d")
    plt.xticks(fontsize=12, color="#6c757d")
    plt.yticks(fontsize=12, color="#6c757d")
    plt.grid(axis="y", linestyle="--", alpha=0.6, color="#b0b0b0")

    # Mostrar leyenda por defecto
    plt.legend(fontsize=12, loc="best", frameon=True, facecolor="white", edgecolor="gray")

    plt.tight_layout()

    # Guardar la gráfica
    output_file = os.path.join(output_dir, f"{model_name.replace(' ', '_')}_comparison.png")
    plt.savefig(output_file)
    plt.close()
    print(f"Gráfica guardada en: {output_file}")




In [ ]:
import os
# Directorios
models_dir = "../files/models"
data_path = "../files/output/data_clean.csv"
output_dir = "../files/plots"

# Iterar sobre los archivos en la carpeta models
for model_file in os.listdir(models_dir):
    if model_file.endswith(".pkl"):  # Procesar solo archivos .pkl
        model_path = os.path.join(models_dir, model_file)
        graf_and_save(
            model_pickle_path=model_path,
            data_path=data_path,
            output_dir=output_dir,
            n_train=n_train
        )